# CDJ

In [1]:
import sys
import os
import gc
import traceback
import json 
import pandas as pd
import numpy as np
import random
import collections
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from statistics import mode, StatisticsError
from collections import Counter 
from typing import List
from operator import itemgetter
from itertools import groupby
import importlib

from datasketch import MinHash

In [2]:
#--------------------#
# sphere package
#--------------------#
sys.path.append("/home/das_share/sphere_class/")
import SpherePackage
from SpherePackage import *
for pkg in [SpherePackage] :
    _ = importlib.reload(pkg)

#--------------------#
# kto package
#--------------------#
sys.path.append("../src/")
import kto_config
import kto_util
import return_pickle
import kto_prep
import CDJ_main

from kto_config import *
from kto_prep import *
from kto_util import *
from return_pickle import *
from CDJ_main import *

for pkg in [kto_config, kto_prep, kto_util, return_pickle] :
    _ = importlib.reload(pkg)

# 1. Data Import

# 2. Data Handling
 - purchse_complete_view

## 1) date_anomaly_detection

In [53]:
df_app_log = ReadFile.read_pickle(e_date='20221113', s_date='20221106')
df_app_log = df_app_log[df_app_log['events'].apply(lambda x: False if len(x)==0 else True)]

In [11]:
df_seq = df_app_log[[KEY_ID_DEVICE, 'sid', 'events']]

# 1) group by sid (using sum since some sid have multi line)
df_seq = EventEngineering.group_sid(df_seq)

# 2) get cdj
df_seq = EventEngineering.get_cdj(df_seq)

# 3) select ids which contain KPI
df_target_seq = EventEngineering.target_id_df(df_seq, KPI = 'subscribe')

In [54]:
CDJ_main.run_cdj(df_target=df_app_log, KPI_event=EVENT_SUBSCRIBE)

(                                      final_cdj_path  cnt  ratio
 0  [sapHome, sapContentDetail, subscribeClicsapCo...   39  38.61
 1  [sapHome, sapContentDetail, subscribeClicsapCo...   11  10.89
 2  [sapHome, homeBanner, sapContentDetail, subscr...    7   6.93
 3             [sapHome, sapContentDetail, subscribe]    6   5.94
 4  [sapHome, homeTopBanner, balanceGameMain, sapC...    4   3.96
 5  [sapHome, drawerMain, eventlMain, eventlBanner...    3   2.97
 6  [sapHome, homeTopBanner, balanceGameMain, bala...    2   1.98,
 510)

In [4]:
#------------------------------------------------#
# 1. log Data import
#------------------------------------------------#
# dates: 10월 한 달
df_app_log = DataImport.read_files(
    input_path = INPUT_PATH + DATA_TYPE + APP_KEY, 
    today = datetime.strptime('20221118', '%Y%m%d') + timedelta(days = 1), 
    dates = (datetime.strptime('20221118', '%Y%m%d') - datetime.strptime('20221110', '%Y%m%d')).days + 1,
    platform_total = True, reduce_memory = True, verbose = False)

In [5]:
def check_event(li, event):
    """check whether purchase is in or not"""
    # 1) row별로 호출하기 -> lambda
    names = []
    for i in range(len(li)):
        # 2) events변수의 딕셔너리 불러오기 -> li
        tmp_name = li[i].get('name')
        # 3) 딕셔너리내의 값 하나씩 빼서 key값이 name인 value가져오기
        names.append(tmp_name)
        
    output = event in names
    
    return output

# datetime data handling
def find_datetime(x):
    if type(x) == pd.Timestamp:
        return True
    else:
        return False

def date_anomaly_detection(df,date_cols):
    if isinstance(date_cols,str):
        df = df.drop(df.loc[df[date_cols].apply(lambda x: find_datetime(x))==False].index,axis=0)
    elif isinstance(date_cols,list):
        select  = df[date_cols].apply(lambda r : all([find_datetime(e) for e in r  ]),axis=1) 
        df=df[select]
    else:
        pass
    return df

In [4]:
# def x_in_y(query, base):
#     """ Returns boolean if given query is included in a base """
#     m = len(query)
#     n = len(base)
#     j = 0
#     i = 0
#    
#     while j < m and i < n: 
#         if query[j] == base[i]:
#             j = j + 1 
#         else: i = i + 1
#    
#     return j == m

In [5]:
# ex) 아래 유저의 경우 30분 이내에 4번이나 왔다갔다하여 동일한 sid로 4개의 row 생성 -> session 단위로 groupby sum을 하는 이유
# df_seq[(df_seq[KEY_ID_DEVICE] == '2d1d1da0-d6ea-48c0-bf75-32d6fb32f676') &(df_seq['sid'] == 'N3UkZE2IRnu_-28bmKwFJg')]

## 2) JsonEvent  to List

In [6]:
# def abstract_events(x):
#     """get events name from list"""
#     path = []
#     for i in x:
#         path.append(i['name'])
#     return path

# def abstract_events_apply (df) :
#     df.loc[:,'events'] = df.events.apply(lambda x: abstract_events(x))
#     df = df.groupby(KEY_ID_DEVICE).agg({'events':'sum'}).reset_index()
#     return df

# 3. CDJ main

## 1) EventEngineering

In [50]:
df_seq = df_app_log[[KEY_ID_DEVICE, 'sid', 'events']]

# 1) group by sid (using sum since some sid have multi line)
df_seq = group_sid(df_seq)

# 2) get cdj
df_seq = get_cdj(df_seq)

# target_id_df
KPI_index = list(map(lambda x: EVENT_SUBSCRIBE in x, df_seq['cdj_path'])) # map(lambda x : [func], [함수에 넣을 list)] . --> output: ex) [True, False, False ..]
df_KPI_ids = pd.DataFrame(df_seq.loc[KPI_index, KEY_ID_DEVICE].unique()).rename(columns = {0:KEY_ID_DEVICE})        ## KPI_index가 True인 애들의 유니크한 uid 추출 -> return to df
df_target = pd.merge(df_seq, df_KPI_ids, on = KEY_ID_DEVICE, how = 'right')

# after target_id_df
df_target['date'] = df_target.events.apply(lambda x : x[0]['date'])     ## 각 행의 첫번째 events의 date으로 'date' column 대체
df_target
df_target_seq = df_target.sort_values('date').reset_index(drop = True).drop(columns = ['date'])     # 날짜순으로 오름차순 정렬
df_target_seq

,uid,sid,events,cdj,cdj_path,date
0,-1BC5PS8COeQTZl5B7ZVpA,cvrsIX0yklMWmMInpD-s4K,"[{'name': 'drawerMain', 'timestamp': 166804826...","[[2022-11-10 11:18:16, sapContentDetail], [202...","[sapContentDetail, contentMain, sapContentDeta...",2022-11-10 11:44:23
1,-1X2nCfS25DMyYWt68qZ4Q,HSm0bR5X15yGXdtmMdVAOn,"[{'name': 'sapContentDetail', 'timestamp': 166...","[[2022-11-10 00:11:41, sapLogin], [2022-11-10 ...","[sapLogin, sapContentDetail, subscribeClicsapC...",2022-11-10 00:11:48
2,-1X2nCfS25DMyYWt68qZ4Q,Q4RNJwRaPn24aCYZX_CPk8,"[{'name': 'sapContentDetail', 'timestamp': 166...","[[2022-11-15 12:53:27, sapContentDetail]]",[sapContentDetail],2022-11-15 12:53:27
3,-1X2nCfS25DMyYWt68qZ4Q,xcSuIQY1AUMeLyChBD16uC,"[{'name': 'drawerMain', 'timestamp': 166817551...","[[2022-11-11 23:05:16, drawerMain], [2022-11-1...","[drawerMain, contentMain, sapContentDetail]",2022-11-11 23:05:16
4,-1cq1Zz0FAr5F64_xUUtEg,NWhiqkSpvjDOFgjzZ_PMsT,"[{'name': 'sapContentDetail', 'timestamp': 166...","[[2022-11-11 11:08:56, sapContentDetail], [202...","[sapContentDetail, sapContentDetail, sapConten...",2022-11-11 11:16:17
...,...,...,...,...,...,...
1688,zqeaVBEHqkbgPCmwx_THrw,jeGGqbNVpuhPVi9_6uFd9x,"[{'name': 'sapContentDetail', 'timestamp': 166...","[[2022-11-15 14:26:54, sapContentDetail], [202...","[sapContentDetail, digitalIdCardIntroMain, dig...",2022-11-15 14:26:54
1689,zqpD3b6UrOA36z8Qyl9Rxg,7ML1cYkWj8AEwLJL4Ha_u2,"[{'name': 'sapContentDetail', 'timestamp': 166...","[[2022-11-10 10:37:25, sapContentDetail], [202...","[sapContentDetail, sapContentDetail, digitalId...",2022-11-10 10:37:25
1690,zsf6kkfSqnWp1YZsZP5Oiw,KrtHUQ3QQjpzvEDMG9zVS6,"[{'name': 'sapContentDetail', 'timestamp': 166...","[[2022-11-11 11:23:17, sapContentDetail], [202...","[sapContentDetail, subscribeClicsapContentShar...",2022-11-11 11:23:17
1691,zv3SO-tD2BXIvkB2u041mg,PrnNTbpA6-8pjDdrof6AYl,"[{'name': 'balanceGameMain', 'timestamp': 1668...","[[2022-11-17 16:43:54, balanceGameMain]]",[balanceGameMain],2022-11-17 16:43:54


,uid,sid,events,cdj,cdj_path
0,Apyr1H1ZXDYDpVrUF2rgdw,H3dH1hKU4EnNZlNjsQ-BH3,"[{'name': 'digitalIdCardIntroMain', 'timestamp...","[[2022-11-10 00:01:21, digitalIdCardIntroMain]...","[digitalIdCardIntroMain, digitalIdCardApplyMai..."
1,5ic7-QTGRCujFOBAarCWXA,2VG7RFaBTE25_EonG5NFPg,"[{'name': 'sapHome', 'timestamp': 166800613752...","[[2022-11-10 00:02:17, sapHome], [2022-11-10 0...","[sapHome, searchMain, bottomNavi, searchMain, ..."
2,EjgrTn-C4Qo9sOMg7PxrVg,M37gorRA0Q2r4qdNb6ulU5,"[{'name': 'digitalIdCardIntroMain', 'timestamp...","[[2022-11-10 00:05:41, digitalIdCardIntroMain]]",[digitalIdCardIntroMain]
3,jCW4S2NUNhFQOSFt-rs24w,o3uU4QP0w6AF6vMuP9w_5i,"[{'name': 'sapContentDetail', 'timestamp': 166...","[[2022-11-10 00:08:05, sapLogin], [2022-11-10 ...","[sapLogin, sapContentDetail, sapContentDetail,..."
4,XLxlXFDF0N0DAuUfIkLRmQ,QtjESHHpKJ6_N3wF-YOxJT,"[{'name': 'sapLogin', 'timestamp': 16680066226...","[[2022-11-10 00:10:22, sapLogin], [2022-11-10 ...","[sapLogin, sapContentDetail, sapContentDetail,..."
...,...,...,...,...,...
1688,xAFQjQRPwaKlTDTXcpoBdw,aHeqdScRyhLrq7-QpZ0hyL,"[{'name': 'sapContentDetail', 'timestamp': 166...","[[2022-11-18 22:59:28, sapContentDetail], [202...","[sapContentDetail, homeSearch, drawerMain, sap..."
1689,YcQuoZf0Lzg9LJcyVoc5sA,I5S5ybGIlRP6TzFZGAbvEj,"[{'name': 'contentMain', 'timestamp': 16687816...","[[2022-11-18 23:27:06, contentMain], [2022-11-...","[contentMain, sapContentDetail, sapContentDeta..."
1690,7T4kyp5rm4w6guUE_sOhkA,qqKP6dP8HugLWeGn2iLE7k,"[{'name': 'sapLogin', 'timestamp': 16687817674...","[[2022-11-18 23:29:27, sapLogin], [2022-11-18 ...","[sapLogin, sapLogin]"
1691,erbRSHut53YCvj-mmLIDrg,3FBhqaVCjlUtijli8XDWNZ,"[{'name': 'sapContentDetail', 'timestamp': 166...","[[2022-11-18 23:31:26, sapContentDetail], [202...","[sapContentDetail, sapHome, searchMain, search..."


In [96]:
# session_to_KPI
KPI = EVENT_SUBSCRIBE

KPI_index = list(map(lambda x: KPI in x, df_target_seq['cdj_path']))
df_seq_KPI = df_target_seq[KPI_index]
session_index = list(map(lambda x: 'sapHome' in x, df_seq_KPI['cdj_path']))
df_seq_KPI = df_seq_KPI.iloc[session_index]

## (2) KPI without sapHome
## 한 세션에서 path가 sapHome이 아닌 경우 --> 비정상적으로 세션이 들어오는 경우이므로 앞의 경로와 이어주어야 함
df_seq_session = df_target_seq[~df_target_seq.index.isin(df_seq_KPI.index)] # dataframe excepted [sapHome to KPI] path
KPI_index = list(map(lambda x: KPI in x, df_seq_session['cdj_path']))  # KPI path index which without sapHome

for sid in df_seq_session.loc[KPI_index, 'sid'] :
    ### (a) get users data  --> sapHome이 찍혀있지 않을 때의 각 uid 당 data 수집
    user_id = df_seq_session.loc[df_seq_session['sid'] == sid][KEY_ID_DEVICE].values[0]
    df_ind = df_seq_session.loc[df_seq_session[KEY_ID_DEVICE] == user_id] # target users dataframe

    ### (b) get idx & final path
    idx = df_seq_session[df_seq_session['sid'] == sid].index.values[0] # get index where KPI(without session) is  --> 가장 먼저 기록된 세션의 idx로 df_ind_idx_reverse에서 가장 마지막값
    df_ind_idx_reverse = list(reversed(df_ind.index.values)) # 유저의 모든 로그 인덱스를 추출하고, 상위 로그로 이동하기위해 순서를 바꿔줌  
    df_ind_idx_reverse = df_ind_idx_reverse[df_ind_idx_reverse.index(idx)+1:] # 목표하고 있는 타겟 로그 이후에 발생한 인덱스는 삭제하고 이전 데이터의 인덱스값만 저장
    final_path = df_target_seq.loc[idx,'cdj_path'] # get path where KPI(without session) is --> 특정 uid sapHome이 없는 로그 기록 중 가장 a

    ### (c) append upper path to final path
    #### 가장 최근 경로서부터 처음 경로까지 거꾸로 for문 진행
    for path_idx in range(len(df_ind_idx_reverse)) :        ## _df_ind_idx_reverse[path_idx] => index 값
        try : 
            upper_path = df_ind.loc[df_ind_idx_reverse[path_idx], 'cdj_path'] 
        except KeyError as e:
            print(e, '{} : there is no upper path'.format(idx))
            break 
        if 'sapHome' in  upper_path :
            final_path = upper_path + final_path
            break
        else : 
            final_path = upper_path + final_path
    df_seq_session.loc[df_seq_session[df_seq_session['sid'] == sid].index.values[0], 'cdj_path'] = final_path

df_seq_KPI = pd.concat([df_seq_KPI, df_seq_session.loc[KPI_index]]).sort_index()
        
## (4) remove KPI without sapHome since it couln't find sapHome event from upper path(or doesn't have upper path) 
session_index = list(map(lambda x: 'sapHome' in x, df_seq_KPI['cdj_path']))
df_seq_KPI = df_seq_KPI.iloc[session_index]

# get_input_unit
df_seq_KPI.loc[:,'cdj_path_reduced'] = df_seq_KPI.cdj_path.apply(lambda x: get_input_unit(x, KPI))

In [106]:
groupby(df_seq_KPI['cdj_path'][1677])

In [8]:
def group_sid(df):
    """
    30분 이내에 background -> foreground로 돌아올시 
    동일한 adid & sid로 2개 이상의 row 생성 가능
    때문에 groupby 후 sum으로 해당 row내의 event값들을 합쳐줌
    """
    df_grouped = df.groupby([KEY_ID_DEVICE,'sid']).agg({'events':'sum'})
    df_grouped = df_grouped.reset_index()
    
    return df_grouped

def create_cdj(x): 
    path = []
    for i in x:
        path.append([i['date'], i['name']])     
    return path

def get_cdj(df):
    df.loc[:,'cdj'] = df.events.apply(lambda x: create_cdj(x))
    df.loc[:,'cdj'] = df.cdj.apply(lambda x: sorted(x, key = lambda y: y[0])) # [date, name] 에서 date기준으로 내부 재정렬
    df.loc[:,'cdj_path'] = df.cdj.apply(lambda x: [c[1] for c in x]) # name만 남기기 ( function(y) for y in x 구조)
    return df

def target_id_df(df, KPI) :
    """extract df which contain only users who have reached the KPI even once"""
    KPI_index = list(map(lambda x: KPI in x, df['cdj_path'])) # map(lambda x : [func], [함수에 넣을 list)]
    df_KPI_ids = pd.DataFrame(df.loc[KPI_index, KEY_ID_DEVICE].unique()).rename(columns = {0:KEY_ID_DEVICE}) # 945명의 구매자
    df_target = pd.merge(df, df_KPI_ids, on = KEY_ID_DEVICE, how = 'right')
    return df_target

def session_to_KPI(df_target_seq, KPI) :
    """
    - extract df which contain only KPI path
    - include KPI path wihtout session in -> append with upper path to find session 
    """
    # (1) KPI with sapHome
    KPI_index = list(map(lambda x: KPI in x, df_target_seq['cdj_path']))
    df_seq_KPI = df_target_seq[KPI_index]
    session_index = list(map(lambda x: 'sapHome' in x, df_seq_KPI['cdj_path']))
    df_seq_KPI = df_seq_KPI.iloc[session_index]

    ## (2) KPI without sapHome
    df_seq_session = df_target_seq[~df_target_seq.index.isin(df_seq_KPI.index)] # dataframe excepted [sapHome to KPI] path
    KPI_index = list(map(lambda x: KPI in x, df_seq_session['cdj_path']))  # KPI path index which without sapHome

    ## find '#seesion' event from upper rows
    for sid in df_seq_session.loc[KPI_index, 'sid'] :
        ### (a) get users data  
        user_id = df_seq_session.loc[df_seq_session['sid'] == sid][KEY_ID_DEVICE].values[0]
        df_ind = df_seq_session.loc[df_seq_session[KEY_ID_DEVICE] == user_id] # target users dataframe

        ### (b) get idx & final path
        idx = df_seq_session[df_seq_session['sid'] == sid].index.values[0] # get index where KPI(without session) is
        df_ind_idx_reverse = list(reversed(df_ind.index.values)) # 유저의 모든 로그 인덱스를 추출하고, 상위 로그로 이동하기위해 순서를 바꿔줌  
        df_ind_idx_reverse = df_ind_idx_reverse[df_ind_idx_reverse.index(idx)+1:] # 목표하고 있는 타겟 로그 이후에 발생한 인덱스는 삭제
        final_path = df_target_seq.loc[idx,'cdj_path'] # get path where KPI(without session) is

        # ### (c) append upper path to final path
        for path_idx in range(len(df_ind_idx_reverse)) :
            try : 
                upper_path = df_ind.loc[df_ind_idx_reverse[path_idx], 'cdj_path'] 
            except KeyError as e:
                print(e, '{} : there is no upper path'.format(idx))
                break 
            if 'sapHome' in upper_path :
                final_path = upper_path + final_path
                break
            else : 
                final_path = upper_path + final_path

        ### (d) replace complete path to KPI path
        df_seq_session.loc[df_seq_session[df_seq_session['sid'] == sid].index.values[0], 'cdj_path'] = final_path
            
    ## (3) merge df_aw_KPI and without sapHome KPI 
    df_seq_KPI = pd.concat([df_seq_KPI, df_seq_session.loc[KPI_index]]).sort_index()
    
    ## (4) remove KPI without sapHome since it couln't find sapHome event from upper path(or doesn't have upper path) 
    session_index = list(map(lambda x: 'sapHome' in x, df_seq_KPI['cdj_path']))
    df_seq_KPI = df_seq_KPI.iloc[session_index]
    
    return df_seq_KPI

def getIndexPositions(listOfElements, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    indexPosList = []
    indexPos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            indexPos = listOfElements.index(element, indexPos) # 첫번째 
                                                               # index(target, 찾는 시작 위치)
            # Add the index position in list
            indexPosList.append(indexPos)
            indexPos += 1
        except ValueError as e:
            break
    
    return indexPosList

def get_input_unit(path, KPI):
    # kpi가 존재하는 가장 최근의 세션만을 사용
    """ Returns part of customer decision journey for sequence analysis """
    if KPI in path:
        # indexing channels from last session to KPI
        # 1) remove all path after KPI
        KPI_index = path.index(KPI) # KPI 위치
        new_path = path[: KPI_index + 1]

        # 2) find the last session (session으로 언제든 돌아올 수 있기에 가장 마지막 session만을 고려)
        session_index = max(getIndexPositions(path, "sapHome")) 
        new_path = new_path[session_index:]
        return new_path

    else:
        session_index = max(getIndexPositions(path, "sapHome"))
        new_path = path[session_index:]
        return new_path

def eventPreprocess(df_seq_KPI, min_length = 3) :
    """
    - simple handling before groupby and sorting
    - 1) remove system variable : '#foreground'
    - 2) remove consecutive action 
    - 3) remove path less than min_length(default >= 3) length 
    """
    ## (1) remove system variable '#foreground'
    df_seq_KPI.loc[:,'cdj_path_reduced'] = df_seq_KPI.cdj_path_reduced.apply(lambda x: [x for x in x if x != '#foreground'])

    ## (2)remove consecutive actions in cdj_path_reduced column : 연속적인 경로 제거  
    df_seq_KPI.loc[:,'cdj_path_reduced'] = df_seq_KPI.cdj_path_reduced.apply(lambda x: list(map(itemgetter(0), groupby(x))))
#     df_seq_KPI.loc[:,'cdj_path_unique'] = df_seq_KPI.cdj_path_reduced.apply(lambda x: unique_path(x))
    
    ## (3) remove path less than minimum length 
    df_seq_KPI['path_len'] = df_seq_KPI.cdj_path_reduced.apply(lambda x : len(x))
    df_seq_KPI = df_seq_KPI.loc[df_seq_KPI['path_len'] >= min_length]

    return df_seq_KPI


In [9]:
def eventEngineer(df, KPI, min_length) :
    # 0. select necessary feature 
    df_seq = df[[KEY_ID_DEVICE, 'sid', 'events']]

    # 1) group by sid (using sum since some sid have multi line)
    df_seq = group_sid(df_seq)

    # 2) get cdj
    df_seq = get_cdj(df_seq)

    # 3) select ids which contain KPI
    df_target_seq = target_id_df(df_seq, KPI)
    df_target_seq['date'] = df_target_seq.events.apply(lambda x : x[0]['date'])
    df_target_seq = df_target_seq.sort_values('date').reset_index(drop = True).drop(columns = ['date']) # arrange by time(to combine KPI without sapHome with upper sapHome)

    # 4) adjusted(remove or combine) to the sapHome and KPI 
    df_seq_KPI = session_to_KPI(df_target_seq, KPI)

    # 5) candidate sequence must start with "sapHome", end with KPI
    df_seq_KPI.loc[:,'cdj_path_reduced'] = df_seq_KPI.cdj_path.apply(lambda x: get_input_unit(x, KPI))
#     df_seq_KPI.loc[:,'cdj_path_unique'] = df_seq_KPI.cdj_path.apply(lambda x: unique_path(x))

    # #6) eventPreprocess
    df_seq_KPI = eventPreprocess(df_seq_KPI, min_length)

    return df_seq, df_seq_KPI

In [10]:
df_seq, df_seq_KPI  = eventEngineer(df_app_log, EVENT_SUBSCRIBE, min_length = 3)

## 2) return_to_seq

In [13]:
def x_in_y(query, base, impurity=1):
    """
    Returns boolean if given query(every_path)
    is included in a base(top_seq) 
    """
    m = len(query)
    n = len(base)
    j = 0
    i = 0
    
    # whether given query included in a base
    while j < m and i < n: 
        if query[j] == base[i]:
            j = j + 1 # 변수가 같으면 top_seq의 다음 경로로 이동
        else: 
            i = i + 1 # 변수가 다르면 path의 다음 경로로 이동
    
    included = j == m # row_path가 top_seq에 모두 속한다면,
                      # 최종적으로 j의 모든 카운트가 올라가고 이는 j의 길이(m)와 같아짐   
    
    # whether given query included in a based under allowed impurity
    ## 아래 두 코드 왜 이렇게 짜여져있는지 이해 xx
    impurity_ratio = len(query) / len(base)
    impurity_accepted = impurity_ratio >= (1 - impurity)
    
    return included & impurity_accepted    

def seq_match_score(candidate, base):
    """ Returns score of candidate sequence for base sequence """
    # A B C D -> G could be matched with A B C -> G or A B D -> G
    # scoring based on how many elements are matched with
    # if the number of matched elements is same, elements close to goal will get higher score
    score_list = []
    elems_of_candidate = list(set(candidate))

    for i in range(len(elems_of_candidate)):
        idx = getIndexPositions(base, elems_of_candidate[i])
        score_list.append(sum(idx))

    output = sum(score_list)
    return output

def find_max_index(li, idx):
    """ Returns index values with biggest numbers in a list when list and index are given """
    # e.g. li = [1,4,3,4], idx = [0,1,3]
    # find_max_index(li, idx) returns [1,3]
    candidate = [li[i] for i in idx]
    max_index = getIndexPositions(candidate, max(candidate))
    output = [idx[i] for i in max_index]
    return output

def assign_seq(top_seq, cdj_path, impurity=1):
    """ Returns which top sequence is matched with given cdj_path"""
    # cdj_path : customer decision journey which means a journey to KPI for a customer, input is a list consisting of many cdj_paths
    # top_seq : most frequent sequences to KPI

    # when top sequence candidates are greater than one,
    # length of each top sequence should be considered
    top_seq_len = [len(x) for x in top_seq]

    # which top_seq is matched with given cdj_path
    output = []

    for i in range(len(cdj_path)):
        # loop every path in a list
        path = cdj_path.iloc[i]
        match_list = [x_in_y(x, path, impurity) for x in top_seq]  # basic_pat가 상위 100개의 path에 포함되는지 여부 확인

        # if there is no top_seq matched with a given path, zero will be designated
        if sum(match_list) == 0:
            output.append(0)

        # if there is only one top_seq matched with a given path, index of that top_seq(+1) will be designated
        elif sum(match_list) == 1:
            output.append(getIndexPositions(match_list, True)[0] + 1) # 매칭된 상위 인덱스에 +1을 더해, 매칭된 인덱스가 없음을 의미하는 0과 다르게 만듦.

        # if there are many top_seq matched with a given path,
        # index of longest top_seq will be designated
        # 'longest' means more elements are matched
        else:
            indexPosList = []
            indexPos = 0
            while True:
                try:
                    # Search for item in list from indexPos to the end of list
                    indexPos = match_list.index(True, indexPos) # 첫번째 
                                                                    # index(target, 찾는 시작 위치)
                    # Add the index position in list
                    indexPosList.append(indexPos)
                    indexPos += 1
                except ValueError as e:
                    break
                
            index = getIndexPositions(match_list, True)
            len_of_matched_seq = [top_seq_len[i] for i in index]
            longest_topseq_idx = getIndexPositions(
                len_of_matched_seq, max(len_of_matched_seq)
            )

            # if there is only one longest top_seq, index of that top_seq will be designated
            if len(longest_topseq_idx) == 1:
                output.append(find_max_index(top_seq_len, index)[0] + 1)

            # if there are two or more longest top_seq,
            # consider matching score
            else:
                # matched longest top sequences
                top_seq_selected = [
                    top_seq[i] for i in find_max_index(top_seq_len, index)
                ]

                # scoring based on matched elements
                score_list = []
                for i in range(len(top_seq_selected)):
                    score_list.append(seq_match_score(top_seq_selected[i], path))

                maxscore_index = getIndexPositions(score_list, max(score_list))

                # indexing top_seq with highest score
                one_top_seq_selected = top_seq_selected[maxscore_index[0]]

                # index of highest score top_seq will be designated
                output.append(top_seq.index(one_top_seq_selected) + 1)

    return output

In [14]:
%%time
def return_top_seq(df_seq_KPI, KPI, impurity=1, top=5): 
    """ Returns selected final top sequences and its count based on the latest 5 weeks dataset(input will be train set) """
    # 1) top_seq contains candidats for top sequence
    # will be rearranged after sequence matching
    vc = df_seq_KPI.cdj_path_reduced.apply(tuple).value_counts() # find top_seq contains condidates for top sequence
    top_seq = pd.DataFrame(vc)[pd.DataFrame(vc)['cdj_path_reduced'] >= 2].index.values.tolist() # extract over 2 value counts
#     top_seq = pd.DataFrame(vc).index.values[0:99].tolist() # extract upper 100 values
#     top_seq = vc.index.values[0:99] # numpy array version
    top_seq = [list(x) for x in top_seq] # list version
    
    # 2) assinged base path to top_seq[*****]
    # output : 조건을 만족하며 base_path가 속하는 top_seq의 index번호들이 list 형태로 출력
    # 애초에 만든 top_seq는 기준일 뿐, assinged_seq를 거치면서 모든 base_path가 다시 top_seq를 기준으로 counting 됨
    assigned_seq = assign_seq(top_seq, df_seq_KPI.cdj_path_reduced, impurity)
    cnt = collections.Counter(assigned_seq) # top_seq에 추가될 index values들의 갯수
    del cnt[0] # remove sequences assigned with zero # Top_seq에 포함되지 못한 애들은 모두 index 0이 되고, 이에 0인 값이 가장 많으므로 0을 제거 
    
    # 3) sorting for extract mod path
    cnt_sorted = sorted(cnt.items(), key = lambda x: x[1], reverse = True) # 쌍으로 이루어진 변수중 두번째값[1]을 기준으로 sort
    selected_top_seq_idx = [x[0] for x in cnt_sorted[0:top]] # 0~99까지중에서 상위 index 5개
    selected_top_seq_cnt = [x[1] for x in cnt_sorted[0:top]] # 상위 index 5개의 빈도
    selected_final_seq = [top_seq[x-1] for x in selected_top_seq_idx]
    
    return selected_final_seq, selected_top_seq_cnt, assigned_seq

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.44 µs


In [15]:
top_seq_result, top_seq_cnt = ReturnToSeq.return_top_seq(df_seq_KPI, EVENT_SUBSCRIBE, impurity=0.5, top=len(df_seq_KPI))

## 3) jaccard_filltering
: grouping similar path

In [16]:
from datasketch import MinHash
def jaccard_filltering(target_list, target_cnt, jaccard_score) :
    """
    jaccard_score = 1   : m1의 요소가 m2와 100% 동일해야만 포함(중복 루트만 포함)
    jaccard_score = 0.5 : m1의 요소가 m2와 50% 동일해야 포함
    jaccard_score = 0.3 : m1의 요소가 m2와 30% 동일하더라도 포함
    => jaccard_score가 낮을 수록 최종 output은 서로 다른 결과가 많아짐(유사한 경로는 포함되어 사라지기에)
    """
    i = 0
    while i < len(target_list) :
        j = i +1
        while j < len(target_list) + 1 :
            m1, m2 = MinHash(), MinHash() 
            try :
                a = target_list[i]
                b = target_list[j]

                for d in a:
                    m1.update(d.encode('utf8'))
                for d in b:
                    m2.update(d.encode('utf8'))

                if m1.jaccard(m2) > jaccard_score :    
                    target_list.remove(target_list[j])
                    target_cnt[i] = target_cnt[i] + target_cnt[j] 
                    target_cnt.remove(target_cnt[j])
                else : 
                    j += 1
            except IndexError as e :
                break
        i += 1    
    return target_list, target_cnt

In [19]:
top_seq_result_fin, top_seq_cnt_fin= jaccard_filltering(top_seq_result, top_seq_cnt, 
                                                         jaccard_score = 0.7) 

## 4) Unique path

In [58]:
def unique_path(x, n_max_unique = 2) : 
    """
    Delete duplicated values
    *It has a risk of deleting essential paths to reach specific paths*
    """
    unique_path_list = []
    for i in x[::-1]:
        if unique_path_list.count(i) >= n_max_unique :
            pass
        else :
            unique_path_list.append(i)
    unique_path_list = unique_path_list[::-1]
    return unique_path_list

## 5) CDJ Main

In [39]:
# 3. CDJ_main
def cdj_main (train, KPI_event,
              impurity=0.5, jaccard_score = 0.5, n_max_unique = 2) : 
    ## 1) eventengineering
    df_seq, df_seq_KPI = eventEngineer(train, KPI_event, min_length = 4)

    ## 2) return to seq 
    top_seq_result, top_seq_cnt = return_top_seq(df_seq_KPI, KPI_event, impurity=0.3, top=len(df_seq_KPI)) 

    ## 3) jaccard_fillter
    top_seq_result_fin, top_seq_cnt_fin  = jaccard_filltering(top_seq_result,
                                                              top_seq_cnt,
                                                              jaccard_score = 0.8) 

#     ## 4) unique_path
#     top_seq_result_fin =  list(map(lambda x : unique_path(x, n_max_unique =2) , top_seq_result_fin))
    
    ## 5) output : dataFrame
    output = pd.DataFrame({'final_cdj_path' : top_seq_result_fin, 
                        'cnt' : top_seq_cnt_fin,
                        'ratio' : list(map(lambda x : round(x/len(df_seq_KPI) *100, 2), top_seq_cnt_fin))}
                     ).sort_values('cnt' , ascending= False).reset_index(drop = True) # top_sequence
    return output

In [20]:
pd.DataFrame({'final_cdj_path' : top_seq_result_fin, 
                        'cnt' : top_seq_cnt_fin,
                        'ratio' : list(map(lambda x : round(x/len(df_seq_KPI) *100, 2), top_seq_cnt_fin))}
                     ).sort_values('cnt' , ascending= False).reset_index(drop = True) # top_sequence

,final_cdj_path,cnt,ratio
0,"[sapHome, searchMain, sapContentDetail, subscr...",92,49.46
1,"[sapHome, searchMain, homeTopBanner, balanceGa...",51,27.42
2,"[sapHome, searchMain, drawerMain, searchMain, ...",11,5.91
3,"[sapHome, sapLogin, travelLetterMain, balanceG...",2,1.08


# 5. Run CDJ

In [60]:
def run_cdj(today, df_target, KPI_event, input_path,
            KEY_ID_DEVICE, dates, impurity=0.3, jaccard_score = 0.8, n_max_unique = 2) :
    # 1. data import
    # train = read_files(today, input_path, dates, True) # 결제 데이터가 많지 않아 android와 ios를 합쳐서 진행
    # train = train.drop(['deeplink', 'referrer'], axis=1)
    total_raw_buy_cnt = sum(train.events.apply(lambda x : check_event(x, KPI_event)))

    # 2. date_anomaly_detection
    train = date_anomaly_detection(train, 'date')

    # 3. cdj_main
    output = cdj_main(today, df_target, KPI_event)
    
    
#     return output
    return output, total_raw_buy_cnt

# EDA & TEST 검증 코드

In [70]:
# ** using module
def run_cdj(df_target, KPI_event, impurity, jaccard_score, n_max_unique = 2) :
    ## total kpi count
    total_raw_kpi = sum(df_target.events.apply(lambda x : AnomalyDetection.check_event(x, KPI_event)))

    ## data anomaly detection
    train = AnomalyDetection.date_anomaly_detection(df_app_log, 'date')

    ## cdj_main
    output = CDJ_main.cdj_main(
                train, KPI_event, impurity=impurity,\
                jaccard_score = jaccard_score, n_max_unique = 2) 
                
    ## return output
    return output, total_raw_kpi


_df_output, total_raw_kpi = run_cdj(df_app_log, KPI_event=EVENT_SUBSCRIBE, impurity=0.3, jaccard_score=0.7)
_df_output

In [51]:
# 2. data preprocessing
## 1) data anomaly detection
train = AnomalyDetection.date_anomaly_detection(df_app_log, 'date')

## 2) eventENgineering
df_seq, df_seq_KPI = EventEngineering.eventEngineer(train, KPI_event, min_length = 3)

In [71]:
# 3. filltering
## 1) return to seq
top_seq_result, top_seq_cnt = ReturnToSeq.return_top_seq(df_seq_KPI, KPI_event, impurity=0.5, top=len(df_seq_KPI))

## 2) jaccard similarity
top_seq_result_fin, top_seq_cnt_fin =\
    JaccardFilltering.jaccard_filltering(top_seq_result, top_seq_cnt, jaccard_score = 0.9)

output = pd.DataFrame({'final_cdj_path' : top_seq_result_fin, 
            'cnt' : top_seq_cnt_fin,
            'ratio' : list(map(lambda x : round(x/len(df_seq_KPI) *100, 2), top_seq_cnt_fin))}
        ).sort_values('cnt' , ascending= False).reset_index(drop = True) # top_sequence

output

,final_cdj_path,cnt,ratio
0,"[sapHome, searchMain, sapContentDetail, subscr...",42,41.58
1,"[sapHome, searchMain, sapContentDetail, subscr...",13,12.87
2,"[sapHome, searchMain, sapContentDetail, subscr...",10,9.90
3,"[sapHome, searchMain, homeBanner, sapContentDe...",8,7.92
4,"[sapHome, sapContentDetail, subscribeClicsapCo...",3,2.97
5,"[sapHome, searchMain, drawerMain, searchMain, ...",3,2.97
6,"[sapHome, searchMain, homeTopBanner, searchMai...",3,2.97
7,"[sapHome, searchMain, homeTopBanner, searchMai...",2,1.98
8,"[sapHome, sapContentDetail, subscribeClicsapCo...",2,1.98


In [72]:
# 4. output: path
print('VC---------------------------------------------')
pd.DataFrame(df_seq_KPI.cdj_path_reduced.apply(tuple).value_counts()).reset_index()

print('return_to_seq---------------------------------------------')
pd.DataFrame({'path' : top_seq_result, 'cnt' : top_seq_cnt})\
            .sort_values(by = 'cnt', ascending= False).reset_index(drop = True)[:10]

print('jaccard---------------------------------------------')
pd.DataFrame({'path' : top_seq_result_fin, 'cnt' : top_seq_cnt_fin})\
            .sort_values(by = 'cnt', ascending= False).reset_index(drop = True)[:10]

VC---------------------------------------------


,index,cdj_path_reduced
0,"(sapHome, searchMain, sapContentDetail, subscr...",15
1,"(sapHome, searchMain, sapContentDetail, subscr...",10
2,"(sapHome, searchMain, sapContentDetail, subscr...",6
3,"(sapHome, searchMain, sapContentDetail, subscr...",4
4,"(sapHome, searchMain, sapContentDetail, subscr...",4
5,"(sapHome, searchMain, homeTopBanner, searchMai...",3
6,"(sapHome, searchMain, homeBanner, sapContentDe...",2
7,"(sapHome, sapContentDetail, subscribeClicsapCo...",2
8,"(sapHome, searchMain, homeTopBanner, searchMai...",2
9,"(sapHome, sapContentDetail, subscribeClicsapCo...",2


return_to_seq---------------------------------------------


,path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscr...",42
1,"[sapHome, searchMain, sapContentDetail, subscr...",13
2,"[sapHome, searchMain, sapContentDetail, subscr...",10
3,"[sapHome, searchMain, homeBanner, sapContentDe...",8
4,"[sapHome, sapContentDetail, subscribeClicsapCo...",3
5,"[sapHome, searchMain, drawerMain, searchMain, ...",3
6,"[sapHome, searchMain, homeTopBanner, searchMai...",3
7,"[sapHome, searchMain, homeTopBanner, searchMai...",2
8,"[sapHome, sapContentDetail, subscribeClicsapCo...",2


jaccard---------------------------------------------


,path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscr...",42
1,"[sapHome, searchMain, sapContentDetail, subscr...",13
2,"[sapHome, searchMain, sapContentDetail, subscr...",10
3,"[sapHome, searchMain, homeBanner, sapContentDe...",8
4,"[sapHome, sapContentDetail, subscribeClicsapCo...",3
5,"[sapHome, searchMain, drawerMain, searchMain, ...",3
6,"[sapHome, searchMain, homeTopBanner, searchMai...",3
7,"[sapHome, searchMain, homeTopBanner, searchMai...",2
8,"[sapHome, sapContentDetail, subscribeClicsapCo...",2


In [33]:
# return_top_seq를 풀어서 살펴봄 ⬇️⬇️
vc = df_seq_KPI.cdj_path_reduced.apply(tuple).value_counts() # find top_seq contains condidates for top sequence
top_seq = pd.DataFrame(vc)[pd.DataFrame(vc)['cdj_path_reduced'] >= 2].index.values.tolist() # extract over 2 value counts
top_seq = [list(x) for x in top_seq] # list version
top_seq_len = [len(x) for x in top_seq]

# which top_seq is matched with given cdj_path
output = []

for i in range(len(df_seq_KPI.cdj_path_reduced)):
# loop every path in a list
    path = df_seq_KPI.cdj_path_reduced.iloc[i]
    match_list = [x_in_y(x, path, 0.5) for x in top_seq]  # basic_pat가 상위 100개의 path에 포함되는지 여부 확인

    # if there is no top_seq matched with a given path, zero will be designated
    if sum(match_list) == 0:
        output.append(0)

    # if there is only one top_seq matched with a given path, index of that top_seq(+1) will be designated
    elif sum(match_list) == 1:
        output.append(getIndexPositions(match_list, True)[0] + 1) # 매칭된 상위 인덱스에 +1을 더해, 매칭된 인덱스가 없음을 의미하는 0과 다르게 만듦.

    # if there are many top_seq matched with a given path,
    # index of longest top_seq will be designated
    # 'longest' means more elements are matched
    else:
        index = getIndexPositions(match_list, True)         ## True의 index 값을 리스트 형태로 반환
        len_of_matched_seq = [top_seq_len[i] for i in index]        ## True였던 path의 길이를 담은 리스트
        longest_topseq_idx = getIndexPositions(
            len_of_matched_seq, max(len_of_matched_seq)             ## 가장 길이가 긴 path가 어디에 위치했는지 index 담은 리스트
        )

        # if there is only one longest top_seq, index of that top_seq will be designated
        if len(longest_topseq_idx) == 1:
            output.append(find_max_index(top_seq_len, index)[0] + 1)

        # if there are two or more longest top_seq,
        # consider matching score
        else:   
            # matched longest top sequences
            top_seq_selected = [
                top_seq[i] for i in find_max_index(top_seq_len, index)          ## 길이 가장 긴 path 중
            ]

            # scoring based on matched elements
            score_list = []
            for i in range(len(top_seq_selected)):
                score_list.append(seq_match_score(top_seq_selected[i], path))   ## 얼마나 일치하는지 score를 매겨

            maxscore_index = getIndexPositions(score_list, max(score_list))     ## 가장 점수가 큰 index를 output으로 반환

            # indexing top_seq with highest score
            one_top_seq_selected = top_seq_selected[maxscore_index[0]]

            # index of highest score top_seq will be designated
            output.append(top_seq.index(one_top_seq_selected) + 1)

assigned_seq = output                                                           ## output의 형태는 걍 index를 담은 거라서 값이 서로 중복될 수 o
cnt = collections.Counter(assigned_seq) # top_seq에 추가될 index values들의 갯수 ++ 위의 output에서 각 index가 몇 개 있는지 카운트해서 딕셔너리로 담음
del cnt[0] # remove sequences assigned with zero # Top_seq에 포함되지 못한 애들은 모두 index 0이 되고, 이에 0인 값이 가장 많으므로 0을 제거 

# 3) sorting for extract mod path
cnt_sorted = sorted(cnt.items(), key = lambda x: x[1], reverse = True) # 쌍으로 이루어진 변수중 두번째값[1]을 기준으로 sort ++ (2: 10) 이렇게 튜플 형태가 리스트로 담겨짐
selected_top_seq_idx = [x[0] for x in cnt_sorted[0:5]] # 상위 index 5개
selected_top_seq_cnt = [x[1] for x in cnt_sorted[0:5]] # 상위 index 5개의 빈도
selected_final_seq = [top_seq[x-1] for x in selected_top_seq_idx]   ## idx+1을 했으니 도로 -1을 해줌

selected_top_seq_cnt

[31, 14, 14, 13, 11]

## Path Process

In [93]:
#### pd.options.display.max_colwidth = 200
# 1. Preprocess
# 1-1. 이벤트 단위 & KPI 존재 하는 row만 선택
print(df_seq_KPI.loc[130, 'cdj_path'])

# 1-2. #Session to KPI & delete system event(#foreground)
print('Preprocess---------------------------------------------')
print(df_seq_KPI.loc[130, 'cdj_path_reduced'])

# 1-3) raw seq
# pd.DataFrame(df_seq_KPI.cdj_path_reduced)
print('Raw Seq(path_reduced)---------------------------------------------')
pd.DataFrame(df_seq_KPI.cdj_path_reduced)

# 3. VC
print('VC---------------------------------------------')
pd.DataFrame(df_seq_KPI.cdj_path_reduced.apply(tuple).value_counts()).reset_index()

# 4. return_to_seq
top_seq_result, top_seq_cnt = return_top_seq(df_seq_KPI, KPI_event, impurity=0.5, top=len(df_seq_KPI)) 
print('return_to_seq---------------------------------------------')
pd.DataFrame({'path' : top_seq_result, 'cnt' : top_seq_cnt})\
            .sort_values(by = 'cnt', ascending= False).reset_index(drop = True)[:10]

# # 5. jaccard 
top_seq_result_fin, top_seq_cnt_fin  = jaccard_filltering(top_seq_result, top_seq_cnt, jaccard_score = 0.7)
print('jaccard---------------------------------------------')
pd.DataFrame({'path' : top_seq_result_fin, 'cnt' : top_seq_cnt_fin})\
            .sort_values(by = 'cnt', ascending= False).reset_index(drop = True)[:10]

['sapHome', 'searchMain', 'digitalIdCardIntroMain', 'sapHome', 'sapContentDetail', 'subscribeClicsapContentShare', 'subscribeClicsapContentShare', 'sapLogin', 'sapContentDetail', 'sapContentDetail', 'subscribeClicsapContentShare', 'subscribe', 'balanceGameMain', 'balanceGame', 'travelLetterMain', 'travelLetterNotLike', 'travelLetterLike', 'sapContentDetail', 'drawerMain', 'promotionMain', 'promotionDetail', 'promotionSearch', 'promotionSearch', 'promotionSearch', 'promotionDetail', 'reserveClick', 'reserveClick', 'sapContentDetail']
Preprocess---------------------------------------------
['sapHome', 'sapContentDetail', 'subscribeClicsapContentShare', 'sapLogin', 'sapContentDetail', 'subscribeClicsapContentShare', 'subscribe']
Raw Seq(path_reduced)---------------------------------------------


,cdj_path_reduced
25,"[sapHome, homeTopBanner, searchMain, topTravel..."
29,"[sapHome, searchMain, homeBanner, searchMain, ..."
30,"[sapHome, searchMain, homeBanner, searchMain, ..."
31,"[sapHome, searchMain, sapContentDetail, subscr..."
38,"[sapHome, searchMain, homeTopBanner, sapConten..."
...,...
5361,"[sapHome, drawerMain, searchMain, travelLetter..."
5368,"[sapHome, searchMain, sapContentDetail, subscr..."
5371,"[sapHome, searchMain, sapContentDetail, subscr..."
5376,"[sapHome, searchMain, bottomNavi, homeSearch, ..."


VC---------------------------------------------


,index,cdj_path_reduced
0,"(sapHome, searchMain, sapContentDetail, subscr...",65
1,"(sapHome, searchMain, sapContentDetail, subscr...",35
2,"(sapHome, searchMain, sapContentDetail, subscr...",28
3,"(sapHome, searchMain, sapContentDetail, subscr...",19
4,"(sapHome, sapContentDetail, subscribeClicsapCo...",15
...,...,...
183,"(sapHome, searchMain, homeTopBanner, searchMai...",1
184,"(sapHome, searchMain, sapContentDetail, subscr...",1
185,"(sapHome, searchMain, homeTopBanner, searchMai...",1
186,"(sapHome, searchMain, drawerMain, searchMain, ...",1


return_to_seq---------------------------------------------


,path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscr...",79
1,"[sapHome, searchMain, sapContentDetail, subscr...",39
2,"[sapHome, searchMain, sapContentDetail, subscr...",38
3,"[sapHome, searchMain, sapContentDetail, subscr...",25
4,"[sapHome, searchMain, homeBanner, searchMain, ...",18
5,"[sapHome, sapContentDetail, subscribeClicsapCo...",16
6,"[sapHome, sapContentDetail, subscribeClicsapCo...",16
7,"[sapHome, searchMain, homeBanner, searchMain, ...",14
8,"[sapHome, searchMain, homeTopBanner, searchMai...",14
9,"[sapHome, searchMain, sapContentDetail, subscr...",13


jaccard---------------------------------------------


,path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscr...",301
1,"[sapHome, searchMain, homeTopBanner, searchMai...",79
2,"[sapHome, searchMain, sapContentDetail, sapLog...",29
3,"[sapHome, sapContentDetail, subscribeClicsapCo...",25
4,"[sapHome, searchMain, drawerMain, searchMain, ...",13
5,"[sapHome, travelLetterMain, balanceGameMain, s...",3
6,"[sapHome, drawerMain, searchMain, homeSearch, ...",2
7,"[sapHome, sapContentDetail, subscribeClicsapCo...",2


## Rank Reliability

In [ ]:
# pd.options.display.max_colwidth = 200
# df_seq.loc[df_seq.events.apply(lambda x : check_event(x, 'sapBuyStore')), ['sid','cdj_path']].tail(50)

In [94]:
df_seq_KPI['cdj_path_reduced_unique'] = df_seq_KPI.cdj_path_reduced.apply(lambda x : list(set(x)))
df_seq_KPI[['cdj_path_reduced','cdj_path_reduced_unique']].head()
# 
"""
특정 이벤트를 실행한 고객의 비율 확인
ex. game전체 비율 
"""
df_event_cnt = pd.DataFrame.from_dict(Counter(df_seq_KPI.cdj_path_reduced_unique.agg(np.sum)), orient='index')
df_event_cnt = df_event_cnt.sort_values(by = 0, ascending = False).reset_index().rename(columns = {'index' : 'event',0 : 'cnt'})
df_event_cnt['ratio'] = round(df_event_cnt['cnt']/df_event_cnt[df_event_cnt['event'] == 'sapHome'].cnt.values[0], 2)
df_event_cnt.head(10)

,cdj_path_reduced,cdj_path_reduced_unique
25,"[sapHome, homeTopBanner, searchMain, topTravel...","[sapContentDetail, topTravelMain, sapLogin, ho..."
29,"[sapHome, searchMain, homeBanner, searchMain, ...","[homeBanner, sapContentDetail, subscribe, subs..."
30,"[sapHome, searchMain, homeBanner, searchMain, ...","[homeBanner, sapContentDetail, subscribe, subs..."
31,"[sapHome, searchMain, sapContentDetail, subscr...","[sapContentDetail, subscribe, subscribeClicsap..."
38,"[sapHome, searchMain, homeTopBanner, sapConten...","[sapContentDetail, homeTopBanner, subscribe, s..."


'\n특정 이벤트를 실행한 고객의 비율 확인\nex. game전체 비율 \n'

,event,cnt,ratio
0,sapContentDetail,487,1.00
1,subscribe,487,1.00
2,sapHome,487,1.00
3,searchMain,439,0.90
4,subscribeClicsapContentShare,371,0.76
5,sapLogin,181,0.37
6,homeTopBanner,99,0.20
7,balanceGameMain,97,0.20
8,homeBanner,82,0.17
9,drawerMain,59,0.12


## Jaccard Valid

In [95]:
# Jaccard_score = 0.7
# 4. return_to_seq
top_seq_result, top_seq_cnt = return_top_seq(df_seq_KPI, KPI_event, impurity=0.5, top=len(df_seq_KPI)) 
print('return_to_seq---------------------------------------------')
pd.DataFrame({'path' : top_seq_result, 'cnt' : top_seq_cnt})\
            .sort_values(by = 'cnt', ascending= False).reset_index(drop = True)[:10]

# # 5. jaccard 
top_seq_result_fin, top_seq_cnt_fin  = jaccard_filltering(top_seq_result, top_seq_cnt, jaccard_score = 0.7)
print('jaccard---------------------------------------------')
pd.DataFrame({'path' : top_seq_result_fin, 'cnt' : top_seq_cnt_fin})\
            .sort_values(by = 'cnt', ascending= False).reset_index(drop = True)[:10]

return_to_seq---------------------------------------------


,path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscr...",79
1,"[sapHome, searchMain, sapContentDetail, subscr...",39
2,"[sapHome, searchMain, sapContentDetail, subscr...",38
3,"[sapHome, searchMain, sapContentDetail, subscr...",25
4,"[sapHome, searchMain, homeBanner, searchMain, ...",18
5,"[sapHome, sapContentDetail, subscribeClicsapCo...",16
6,"[sapHome, sapContentDetail, subscribeClicsapCo...",16
7,"[sapHome, searchMain, homeBanner, searchMain, ...",14
8,"[sapHome, searchMain, homeTopBanner, searchMai...",14
9,"[sapHome, searchMain, sapContentDetail, subscr...",13


jaccard---------------------------------------------


,path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscr...",301
1,"[sapHome, searchMain, homeTopBanner, searchMai...",79
2,"[sapHome, searchMain, sapContentDetail, sapLog...",29
3,"[sapHome, sapContentDetail, subscribeClicsapCo...",25
4,"[sapHome, searchMain, drawerMain, searchMain, ...",13
5,"[sapHome, travelLetterMain, balanceGameMain, s...",3
6,"[sapHome, drawerMain, searchMain, homeSearch, ...",2
7,"[sapHome, sapContentDetail, subscribeClicsapCo...",2


In [96]:
# Jaccard_score = 0.8
# 4. return_to_seq
top_seq_result, top_seq_cnt = return_top_seq(df_seq_KPI, KPI_event, impurity=0.5, top=len(df_seq_KPI)) 
print('return_to_seq---------------------------------------------')
pd.DataFrame({'path' : top_seq_result, 'cnt' : top_seq_cnt})\
            .sort_values(by = 'cnt', ascending= False).reset_index(drop = True)[:10]

# # 5. jaccard 
top_seq_result_fin, top_seq_cnt_fin  = jaccard_filltering(top_seq_result, top_seq_cnt, jaccard_score = 0.8)
print('jaccard---------------------------------------------')
pd.DataFrame({'path' : top_seq_result_fin, 'cnt' : top_seq_cnt_fin})\
            .sort_values(by = 'cnt', ascending= False).reset_index(drop = True)[:10]

return_to_seq---------------------------------------------


,path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscr...",79
1,"[sapHome, searchMain, sapContentDetail, subscr...",39
2,"[sapHome, searchMain, sapContentDetail, subscr...",38
3,"[sapHome, searchMain, sapContentDetail, subscr...",25
4,"[sapHome, searchMain, homeBanner, searchMain, ...",18
5,"[sapHome, sapContentDetail, subscribeClicsapCo...",16
6,"[sapHome, sapContentDetail, subscribeClicsapCo...",16
7,"[sapHome, searchMain, homeBanner, searchMain, ...",14
8,"[sapHome, searchMain, homeTopBanner, searchMai...",14
9,"[sapHome, searchMain, sapContentDetail, subscr...",13


jaccard---------------------------------------------


,path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscr...",267
1,"[sapHome, searchMain, homeTopBanner, searchMai...",63
2,"[sapHome, sapContentDetail, subscribeClicsapCo...",25
3,"[sapHome, searchMain, homeTopBanner, balanceGa...",20
4,"[sapHome, sapContentDetail, subscribeClicsapCo...",16
5,"[sapHome, searchMain, sapContentDetail, sapLog...",15
6,"[sapHome, searchMain, homeBanner, sapContentDe...",14
7,"[sapHome, searchMain, drawerMain, searchMain, ...",13
8,"[sapHome, searchMain, homeBanner, searchMain, ...",12
9,"[sapHome, drawerMain, searchMain, homeSearch, ...",4


In [97]:
# Jaccard_score = 0.9
# 4. return_to_seq
top_seq_result, top_seq_cnt = return_top_seq(df_seq_KPI, KPI_event, impurity=0.5, top=len(df_seq_KPI)) 
print('return_to_seq---------------------------------------------')
pd.DataFrame({'path' : top_seq_result, 'cnt' : top_seq_cnt})\
            .sort_values(by = 'cnt', ascending= False).reset_index(drop = True)[:10]

# # 5. jaccard 
top_seq_result_fin, top_seq_cnt_fin  = jaccard_filltering(top_seq_result, top_seq_cnt, jaccard_score = 0.9)
print('jaccard---------------------------------------------')
pd.DataFrame({'path' : top_seq_result_fin, 'cnt' : top_seq_cnt_fin})\
            .sort_values(by = 'cnt', ascending= False).reset_index(drop = True)[:10]

return_to_seq---------------------------------------------


,path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscr...",79
1,"[sapHome, searchMain, sapContentDetail, subscr...",39
2,"[sapHome, searchMain, sapContentDetail, subscr...",38
3,"[sapHome, searchMain, sapContentDetail, subscr...",25
4,"[sapHome, searchMain, homeBanner, searchMain, ...",18
5,"[sapHome, sapContentDetail, subscribeClicsapCo...",16
6,"[sapHome, sapContentDetail, subscribeClicsapCo...",16
7,"[sapHome, searchMain, homeBanner, searchMain, ...",14
8,"[sapHome, searchMain, homeTopBanner, searchMai...",14
9,"[sapHome, searchMain, sapContentDetail, subscr...",13


jaccard---------------------------------------------


,path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscr...",128
1,"[sapHome, searchMain, sapContentDetail, subscr...",62
2,"[sapHome, searchMain, homeBanner, searchMain, ...",45
3,"[sapHome, searchMain, homeTopBanner, searchMai...",41
4,"[sapHome, searchMain, homeTopBanner, balanceGa...",33
5,"[sapHome, searchMain, sapContentDetail, subscr...",25
6,"[sapHome, sapContentDetail, subscribeClicsapCo...",25
7,"[sapHome, sapContentDetail, subscribeClicsapCo...",16
8,"[sapHome, searchMain, homeBanner, sapContentDe...",14
9,"[sapHome, searchMain, sapContentDetail, sapLog...",13


In [98]:
# Jaccard_score = 0.95
# 4. return_to_seq
top_seq_result, top_seq_cnt = return_top_seq(df_seq_KPI, KPI_event, impurity=0.5, top=len(df_seq_KPI)) 
print('return_to_seq---------------------------------------------')
pd.DataFrame({'path' : top_seq_result, 'cnt' : top_seq_cnt})\
            .sort_values(by = 'cnt', ascending= False).reset_index(drop = True)[:10]

# # 5. jaccard 
top_seq_result_fin, top_seq_cnt_fin  = jaccard_filltering(top_seq_result, top_seq_cnt, jaccard_score = 0.95)
print('jaccard---------------------------------------------')
pd.DataFrame({'path' : top_seq_result_fin, 'cnt' : top_seq_cnt_fin})\
            .sort_values(by = 'cnt', ascending= False).reset_index(drop = True)

return_to_seq---------------------------------------------


,path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscr...",79
1,"[sapHome, searchMain, sapContentDetail, subscr...",39
2,"[sapHome, searchMain, sapContentDetail, subscr...",38
3,"[sapHome, searchMain, sapContentDetail, subscr...",25
4,"[sapHome, searchMain, homeBanner, searchMain, ...",18
5,"[sapHome, sapContentDetail, subscribeClicsapCo...",16
6,"[sapHome, sapContentDetail, subscribeClicsapCo...",16
7,"[sapHome, searchMain, homeBanner, searchMain, ...",14
8,"[sapHome, searchMain, homeTopBanner, searchMai...",14
9,"[sapHome, searchMain, sapContentDetail, subscr...",13


jaccard---------------------------------------------


,path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscr...",128
1,"[sapHome, searchMain, sapContentDetail, subscr...",62
2,"[sapHome, searchMain, homeBanner, searchMain, ...",45
3,"[sapHome, searchMain, homeTopBanner, searchMai...",32
4,"[sapHome, searchMain, sapContentDetail, subscr...",25
5,"[sapHome, sapContentDetail, subscribeClicsapCo...",25
6,"[sapHome, searchMain, homeTopBanner, balanceGa...",20
7,"[sapHome, sapContentDetail, subscribeClicsapCo...",16
8,"[sapHome, searchMain, homeBanner, sapContentDe...",14
9,"[sapHome, searchMain, sapContentDetail, sapLog...",13


#### ratio 합이 100%가 되지 않는 이유
빈도수 기준 상위 99개에 재포함하는 과정에서 불순도(Inpurity)값을 넘어서지 못한 경로들은 제외되고 나머지 경로로만 분석이 진행됨.<br>
따라서 남은 분석 경로는 전체 구매자보다 적으나, "전체구매자중에서 해당 경로를 거친 사용자"라는 의미를 살리기 위해<br> 분모는 전체구매자로 설정함 모수는 전체 구매자로 설정함.


## 1) impurity

In [ ]:
# 2) return to seq 
result_01, selected_top_seq_cnt_01 = return_top_seq(df_seq_KPI, KPI_event, impurity=0.1, top=50) 
result_03, selected_top_seq_cnt_03 = return_top_seq(df_seq_KPI, KPI_event, impurity=0.3, top=50) 
result_05, selected_top_seq_cnt_05 = return_top_seq(df_seq_KPI, KPI_event, impurity=0.5, top=50) 
result_07, selected_top_seq_cnt_07 = return_top_seq(df_seq_KPI, KPI_event, impurity=0.7, top=50) 

n_list = 7
for i in range(n_list) : 
    print('{}위 ============================================================'.format(i+1))
    print(selected_top_seq_cnt_01[i], result_01[i])
    print(selected_top_seq_cnt_03[i], result_03[i])
    print(selected_top_seq_cnt_05[i], result_05[i])
    print(selected_top_seq_cnt_07[i], result_07[i])
    print('\n')

1위 ============================================================
65 ['sapHome', 'searchMain', 'sapContentDetail', 'subscribeClicsapContentShare', 'subscribe']
73 ['sapHome', 'searchMain', 'sapContentDetail', 'subscribeClicsapContentShare', 'subscribe']
79 ['sapHome', 'searchMain', 'sapContentDetail', 'subscribeClicsapContentShare', 'subscribe']
82 ['sapHome', 'searchMain', 'sapContentDetail', 'subscribeClicsapContentShare', 'subscribe']


2위 ============================================================
35 ['sapHome', 'searchMain', 'sapContentDetail', 'subscribeClicsapContentShare', 'sapLogin', 'sapContentDetail', 'subscribeClicsapContentShare', 'subscribe']
37 ['sapHome', 'searchMain', 'sapContentDetail', 'subscribeClicsapContentShare', 'sapLogin', 'sapContentDetail', 'subscribeClicsapContentShare', 'subscribe']
39 ['sapHome', 'searchMain', 'sapContentDetail', 'subscribeClicsapContentShare', 'sapContentDetail', 'subscribeClicsapContentShare', 'subscribe']
39 ['sapHome', 'searchMain', 'sa

## 2) jaccard score

In [ ]:
# pd.options.display.max_colwidth = 200

## 4. jaccard_fillter
result_03, selected_top_seq_cnt_03 = return_top_seq(df_seq_KPI, KPI_event, impurity=0.3, top=50) 
target_list = result_03
target_cnt = selected_top_seq_cnt_03
target_list_final_j07, target_cnt_final_j07 = jaccard_filltering(target_list,target_cnt, 
                                                         jaccard_score = 0.7) 
pd.DataFrame({'final_cdj_path' : target_list_final_j07,'cnt' : target_cnt_final_j07}).head()

## 4. jaccard_fillter
result_03, selected_top_seq_cnt_03 = return_top_seq(df_seq_KPI, KPI_event, impurity=0.3, top=50) 
target_list = result_03
target_cnt = selected_top_seq_cnt_03
target_list_final_j05, target_cnt_final_j05 = jaccard_filltering(target_list,target_cnt, 
                                                         jaccard_score = 0.5) 
pd.DataFrame({'final_cdj_path' : target_list_final_j05,'cnt' : target_cnt_final_j05}).head()


## 4. jaccard_fillter
result_03, selected_top_seq_cnt_03 = return_top_seq(df_seq_KPI, KPI_event, impurity=0.3, top=50) 
target_list = result_03
target_cnt = selected_top_seq_cnt_03
target_list_final_j03, target_cnt_final_j03 = jaccard_filltering(target_list,target_cnt, 
                                                         jaccard_score = 0.3) 
pd.DataFrame({'final_cdj_path' : target_list_final_j03, 'cnt' : target_cnt_final_j03}).head()



,final_cdj_path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscribeClicsapContentShare, subscribe]",267
1,"[sapHome, sapContentDetail, subscribeClicsapContentShare, sapLogin, sapContentDetail, subscribeClicsapContentShare, subscribe]",24
2,"[sapHome, searchMain, homeTopBanner, searchMain, balanceGameMain, sapContentDetail, subscribe]",73
3,"[sapHome, searchMain, sapContentDetail, sapLogin, sapContentDetail, subscribe]",23
4,"[sapHome, searchMain, drawerMain, searchMain, eventlMain, eventlBanner, sapContentDetail, subscribeClicsapContentShare, subscribe]",11


,final_cdj_path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscribeClicsapContentShare, subscribe]",390
1,"[sapHome, homeTopBanner, searchMain, balanceGameMain, balanceGame, sapLogin, balanceGameMain, sapContentDetail, subscribe]",12
2,"[sapHome, travelLetterMain, balanceGameMain, sapContentDetail, subscribe]",3


,final_cdj_path,cnt
0,"[sapHome, searchMain, sapContentDetail, subscribeClicsapContentShare, subscribe]",405


## 3) unique

In [ ]:
len(df_seq_KPI)